In [ ]:
import pandas as pd
import numpy as np
import warnings
from datetime import datetime, timedelta
warnings.filterwarnings('ignore')

# Human Resource / People analytics data.
Project for data cleaning and data visualisation. Read disclaimer attributed to the original author for this project.

# Data cleaning

## Extraction Phase

There is a total of 5 data in which one of them is about survey. The remaining 4 data is considered to essential for the overall analysis while the survey data will be separately used for engagement analysis. Both combined will create People Analytics.

I identified the 4 data as listed:
- company data → `2021.06_COL_2021.txt`
- job_detail data → `2021.06_job_profile_mapping.txt`
- full data → `CompanyData.txt`
- demographic data → `Diversity.txt`

In addition with survey data → `EngagementSurvey.txt`

In [ ]:
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt",'a') as f:
        f.write(timestamp+', '+message+'\n')
    print(message)

### Importing first 4 data

In [ ]:
import time

In [ ]:
log('Extracting Data ...')
start_time = time.time()

In [ ]:
company_data = pd.read_csv('data/2021.06_COL_2021.txt', sep='\t')
company_data.head()

In [ ]:
job_data = pd.read_csv('data/2021.06_job_profile_mapping.txt', sep='\t')
job_data.head()

In [ ]:
full_data = pd.read_csv('data/CompanyData.txt', sep='\t', encoding='utf_16_le')
full_data.head()

In [ ]:
demographic_data = pd.read_csv('data/Diversity.txt', sep='\t')
demographic_data.head()

In [ ]:
survey_data = pd.read_csv('data/EngagementSurvey.txt', sep='\t')
survey_data.head()

In [ ]:
log('Extraction done in --- %s seconds ---' % (time.time()-start_time))

# Transformation Phase

In [ ]:
log('Transforming Data ...')
start_time = time.time()

Let's first find out how many data we have in each table

In [ ]:
print('The company data has {0} columns and {1} rows'.format(company_data.shape[1],company_data.shape[0]))
print('The job data has {0} columns and {1} rows'.format(job_data.shape[1],job_data.shape[0]))
print('The full data has {0} columns and {1} rows'.format(full_data.shape[1],full_data.shape[0]))
print('The demographic data has {0} columns and {1} rows'.format(demographic_data.shape[1],demographic_data.shape[0]))
print('The survey data has {0} columns and {1} rows'.format(survey_data.shape[1],survey_data.shape[0]))

## Company Data

In [ ]:
company_data

We are going to remove `COL Amount` because it is not relevant to analysis.

In [ ]:
company_data.drop('COL Amount', axis=1, inplace=True)

The table stores office details, it does not contain any meaningful features, so I decided to remove `COL Amount` for now.

## Job data

In [ ]:
job_data.head(10)

In [ ]:
job_data.describe(include='all')

### Converting `Compensation` into float data type

In [ ]:
job_data.rename(columns={' Compensation ':'Compensation'}, inplace=True)
job_data['Compensation']=job_data['Compensation'].str.strip().str.replace(',','')
job_data['Compensation']=pd.to_numeric(job_data['Compensation'])

In [ ]:
job_data.head()

In [ ]:
job_data.describe(include='all')

In [ ]:
job_data.isna().sum()

The job details data does not have any null values. At the current stage, I only need to convert `Compensation` data type.

Changes note: Adding validation discovered in normalization

Checking if job profile in full_data matches job profile in job_data

In [ ]:
job_data.rename(columns={'Bonus %':'Bonus_pct', 'Level':'level'}, inplace=True)
col_to_drop = list(job_data.columns)
col_to_drop

col_to_drop.remove('Compensation')

In [ ]:
job_copy = job_data.copy()

In [ ]:
job_data.equals(job_copy)

In [ ]:
job_copy=job_copy[['Job_Profile']]
job_copy.head()

In [ ]:
job_copy = job_copy.merge(full_data[col_to_drop], on='Job_Profile', how='left').drop_duplicates()
print(job_copy.shape)
job_copy.reset_index(drop=True,inplace=True)
job_copy

Obviously, there are duplicated values of job profile. `Salary` was omitted here because it does not depend on job profile. If we had included `Salary`, there will be **750** unique job profiles. We are going to add the new job profiles created above by creating new job_profile (ids). This will not remove the original job profile, instead we add more job profiles. Originally we have 54 job profiles, now we will ad 6 more.

Creating new job profiles

In [ ]:
job_copy.drop('Job_Profile',axis=1, inplace=True)
job_copy['Job_Profile']=['JP_'+str(i) for i in range(1000,1000+job_copy.shape[0])]
job_copy

In [ ]:
job_data[job_data.duplicated(subset='Job_Profile')] # Check if there are still duplicates

Add compensation back

In [ ]:
col_to_drop.append('Compensation')
col_to_drop.remove('Job_Profile')
col_to_drop

In [ ]:
col_on = col_to_drop.copy()
col_on.remove('Compensation')
job_copy = job_copy.merge(job_data[col_to_drop], on=col_on, how='left')
job_copy.head()

In [ ]:
full_data.drop('Job_Profile', axis=1, inplace=True)

In [ ]:
col_to_drop.remove('Compensation')

In [ ]:
full_data=full_data.merge(job_copy, on=col_to_drop, how='left')
full_data.head(10)

In [ ]:
full_data.columns

Validated Job profile for job_data, created new job profiles and fixing it inside full_data.

## Full data

In [ ]:
full_data.head(10)

In [ ]:
full_data['Notes']

Unseen Columns

In [ ]:
full_data.head().iloc[:,5:]

In [ ]:
full_data.describe(include='all').iloc[:,:13]

In [ ]:
full_data.describe(include='all').iloc[:,13:]

### Data Validation

We are going to check whether there is any values that violate data integrity. First, we will check the age of the employees.  
I am going to do a general check first by checking the distribution of employees' age. This can be found out in the descriptive statistics above and below for detailed view.

In [ ]:
full_data.describe(include='all').loc[:,'Age']

Check complete. The minimum age is 19 and the maximum age is 90. This looks normal except probably age 90 where it could be the CEO or someone with high position.  

Now, we are going to check the termination date. We are going to do a simple check where the value cannot be more than the today date. However, this value actually can be true because some employee might give notice or had a plan to quit the company in near future (all back to company rules).

Notice that the termination_date includes 12/12/2999 which indicate that the employee has not terminated yet. For example:

In [ ]:
full_data.loc[full_data['Termination_Date']=='12/12/2999'].head()

For data analysis purpose, replacing it with `NaN` will be more meaningful.  
Now we can check for termination date that is past today date.

In [ ]:
full_data['Start_Date']=pd.to_datetime(full_data['Start_Date'])
full_data['Termination_Date']=pd.to_datetime(full_data['Termination_Date'], errors='coerce')
full_data.dtypes

In [ ]:
from datetime import datetime,timedelta

full_data.loc[full_data['Termination_Date']>datetime.today(),'Start_Date':]

It turns out there is a lot of rows with termination date violating data integrity **(124 rows)**. Obviously this can or cannot happen in real-world scenario (depends on company rules). The company might allow for employee to give for example 1 month notice period. But, more than 1 month is a little unrealistic. So, for this project purposes, we are going to convert the termination date to today's date. 

While we are at it, might have to check if there are any data that has start date after termination date.

In [ ]:
full_data.loc[full_data['Termination_Date']<full_data['Start_Date'],'Start_Date':]

Check complete.

In [ ]:
# Converting termination date to today's date
full_data['Termination_Date'].loc[full_data['Termination_Date']>datetime.today()]=datetime.today()
full_data.iloc[189,12:]

Dropping `Notes` column.

In [ ]:
full_data.drop('Notes', axis=1, inplace=True)

Adding `start_year` and `termination_year` as a column

In [ ]:
full_data['start_year']=full_data['Start_Date'].dt.strftime('%Y')
full_data['termination_year']=full_data['Termination_Date'].dt.strftime('%Y')
full_data.tail().iloc[:,12:]

In [ ]:
full_data['termination_year'].fillna(0, inplace=True) # Fill null values in termination year with 0
full_data = full_data.astype({'start_year':int,'termination_year':int})

Adding `tenure_months` and `tenure_years` to data by identifying whether employee still active or not

Before we add, we are going to check if active status is true to termination date i.e. checking if there is any employee with active status true having terminated.

In [ ]:
full_data.loc[(full_data['Active Status']==1) & (full_data['termination_year']>0)]

It shows that there is 700 rows with active status still true for terminated employee. We are going to turn the true value into false.

In [ ]:
full_data['Active Status'].loc[(full_data['Active Status']==1) & (full_data['termination_year']>0)]=0
full_data.loc[(full_data['Active Status']==1) & (full_data['termination_year']>0)]

Converting complete, now adding columns.

First, extract the difference in days (we are going to drop this in the end) to make calculations easier for getting months and years.

In [ ]:
full_data['diff_in_days']=0

full_data['diff_in_days'].loc[full_data['Active Status']==0]=(full_data['Termination_Date']-full_data['Start_Date'])
full_data['diff_in_days'].loc[full_data['Active Status']==1]=(datetime.today()-full_data['Start_Date'])

full_data['tenure_months']=full_data['diff_in_days']/timedelta(days=30)
full_data['tenure_years']=full_data['diff_in_days']/timedelta(days=365)
full_data.tail(15).loc[:,'Start_Date':]

Checking other parts of the data

In [ ]:
full_data.iloc[4960:,12:]

In [ ]:
full_data.drop('diff_in_days', axis=1, inplace=True)

#### Cleaning inconsistent state abbreviation

In [ ]:
list(set(full_data['State']))[:10]

We can see from above example that some state is not abbreviated defeating the purpose of `StateFull` column. We are going to replace the full name in `State` to abbreviated code.

Checking if there are any states beside US States

In [ ]:
print('Null values for states beside US =',full_data.loc[full_data['Country']!='US']['State'].isna().sum())
full_data.loc[full_data['Country']!='US']['State']

We are going to webscrape the table for abbreviation US states.

In [ ]:
url='https://www23.statcan.gc.ca/imdb/p3VD.pl?Function=getVD&TVD=53971'

In [ ]:
import requests
from bs4 import BeautifulSoup

html_data = requests.get(url).text
html_data

In [ ]:
soup = BeautifulSoup(html_data, 'html5lib')

In [ ]:
table = soup.find_all('table')[0] # The located table

In [ ]:
table_dict = {}

table_dict['StateFull']=[]
table_dict['Alpha code']=[]

i = 0
for row in table.find_all('td'): # Iterate rows
    if i%3==0 and i!=0: # There are 3 columns in 'td' element, we need to iterate every 3 column to get to the new row
        i=0 # reset i
    # Extract state name
    if i==0: 
        table_dict['StateFull'].append(' '.join(row.contents))
    # Extract state code
    if i==2:
        table_dict['Alpha code'].append(' '.join(row.contents))
    i+=1

In [ ]:
state_df = pd.DataFrame(table_dict)
state_df.head()

In [ ]:
full_data=full_data.merge(state_df, on='StateFull', how='left')
full_data.head()

Adding Washington DC state manually as DC. This is because there are 2 washington's in the data, Washington DC and Washington. The correct state name for Washington DC is District of Columbia, so we are going to change that as well.

In [ ]:
full_data.loc[full_data['Country']!='US']['Alpha code'] # Check whether there are still the same number of null values

In [ ]:
full_data.drop('State', axis=1, inplace=True)
full_data.rename(columns={'Alpha code':'State_code'}, inplace=True)
full_data.head()

In [ ]:
full_data['State_code'].loc[full_data['StateFull']=='Washington DC']='DC'
full_data['StateFull'].loc[full_data['State_code']=='DC']='District of Columbia'
full_data.loc[full_data['State_code']=='DC']

In [ ]:
# Check whether state_code is an abbreviation now
set(full_data['State_code'])

Update age to reflect current year

In [ ]:
full_data['DOB']=pd.to_datetime(full_data['DOB'])
full_data['Age']=round((datetime.today()-full_data['DOB'])/timedelta(days=365))
full_data.head()

Converting all currencies except USD to USD using exchange rates API.

In [ ]:
url = 'https://v6.exchangerate-api.com/v6/bbcdd4717012c0fb7b20f062/latest/USD'
jsondata = requests.get(url).json()
ex_rates = jsondata['conversion_rates']
ex_rates

In [ ]:
full_data['Salary(USD)'] = full_data.apply(lambda x: x.Salary/ex_rates[x.Currency], axis=1)
full_data[['CountryFull','Salary', 'Compensation','Currency', 'Salary(USD)']].loc[full_data['Currency']=='GBP'].head()

In [ ]:
full_data.drop('Salary', axis=1, inplace=True)

The full data (or main data) is a join of the other 3 data (except survey). It includes all details about the employee.  

In [ ]:
log("Transformation that were done in full data {0} :".format(datetime.today()))
log("- Replacing termination date value of 12/12/2999 to NaN")
log("- Converting violating termination date values to today's date")
log("- Drop 'notes' column")
log("- Adding start_year and termination_year")
log("- Inactivating status for terminated employees")
log("- Adding tenure_year and tenure_months")
log("- Cleaning state column")
log("- Updating age values")
log("- Converting salary to USD")

## Demographic Data

In [ ]:
demographic_data.head(10)

In [ ]:
demographic_data.isna().sum()

In [ ]:
demographic_data.describe(include='all')

Investigating null values in Race/Ethnicity column

In [ ]:
set(demographic_data['Race/Ethnicity'])

We have 9 unique ethnicities in the data. For `NaN` values, I will treat is as `Prefer not to say` as it will be more meaningful for our data analysis purposes.

In [ ]:
demographic_data.loc[demographic_data['Race/Ethnicity'].isna()].head(5)

In [ ]:
demographic_data['Race/Ethnicity'].fillna('Prefer not to say', inplace=True)
demographic_data.loc[demographic_data['Race/Ethnicity'].isna()]

This also removes all null values from demographic data.

## Survey Data

In [ ]:
survey_data.head(10)

In [ ]:
survey_data.describe(include='all').iloc[:,:10]

In [ ]:
survey_data.describe(include='all').iloc[:,10:]

The survey is a rating with a range of 1-5 with 5 being the highest. There is 19 questions for the survey. Two additional questions was for feedback towards TheCompany. I am going to extract the `quarter` value from `Survey` column.

In [ ]:
survey_data['survey_quarter']=survey_data['Survey'].str[-2:]
survey_data.head().iloc[:,16:]

Let's look at the null values

In [ ]:
survey_data.isna().sum()

All rows for the feedback columns i.e. `What does TheCompany do well?` and `What can TheCompany improve?` are null. We are going to remove those columns because at this stage it does not give any information.

In [ ]:
survey_data.drop(['What does TheCompany do well?','What can TheCompany improve?'], axis=1,
                inplace=True)
survey_data.head()

In [ ]:
log('Transformation done in --- %s seconds ---' % (time.time()-start_time))

We just dropped the two empty columns and add survey time `quarter` to the data.  
Now, let's load the data into a new dataset.

## Loading Phase

In [ ]:
log('Loading Data ...')
start_time = time.time()

In [ ]:
company_data.to_csv('cleanData/company_details.csv', index=False)
job_copy.to_csv('cleanData/job_details.csv', index=False)
full_data.to_csv('cleanData/main_data.csv', index=False)
demographic_data.to_csv('cleanData/employee_details.csv', index=False)
survey_data.to_csv('cleanData/survey_data.csv', index=False)

In [ ]:
log('Loading done in --- %s seconds ---' % (time.time()-start_time))
log('\n')